In [1]:
!pip install transformers[tf,torch,sentencepiece,vision,optuna,sklearn,onnxruntime]==4.11.3
!pip install transformers
!pip install datasets[audio]==1.16.1
!pip install seqeval
!pip install spacy
!pip install es_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pickle
from pathlib import Path
import re
import torch
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizerFast #DistilBertTokenizerFast
from transformers import RobertaForTokenClassification, Trainer, TrainingArguments #DistilBertForTokenClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from torch import nn
import numpy as np
#!pip install nltk
from collections import Counter
from collections import defaultdict
import nltk
nltk.download('punkt')
from datasets import Dataset, DatasetDict, Features, Value, ClassLabel, Sequence
torch.cuda.is_available()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
!nvidia-smi

Fri Sep 16 09:16:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Cargar CLARA-DM

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
os.listdir('/content/drive/MyDrive/Colab-Notebooks/TFM-HIPE/')

['Diario_de_Madrid_1_4_1800.tsv.iob',
 'Diario_de_Madrid_1_12_1800.tsv.iob',
 'Diario_de_Madrid_1_5_1820.tsv.iob',
 'Diario_de_Madrid_1_3_1800.tsv.iob',
 'Diario_de_Madrid_1_7_1821.tsv.iob',
 'xlm_r_clara_3_eps.ipynb']

In [10]:
docs = [doc for doc in os.listdir('/content/drive/MyDrive/Colab-Notebooks/TFM-HIPE/') if doc.endswith('iob') ]

In [11]:
docs

['Diario_de_Madrid_1_4_1800.tsv.iob',
 'Diario_de_Madrid_1_12_1800.tsv.iob',
 'Diario_de_Madrid_1_5_1820.tsv.iob',
 'Diario_de_Madrid_1_3_1800.tsv.iob',
 'Diario_de_Madrid_1_7_1821.tsv.iob']

In [12]:
train = docs[2:]
val = docs[0]
test = docs[1]

In [13]:
# encadenar los csv en uno solo
columns= ['Token', 'Tag']
# nota: esta celda solo se puede ejecutar una vez para que no se sobreescriba el dataframe final "df_fin"
for i in train:
  df_current = pd.read_csv('/content/drive/MyDrive/Colab-Notebooks/TFM-HIPE/' + str(i), sep='\t', names=columns, encoding='latin1')
  print(df_current)
  try:
    df_fin__ = df_fin__.append(df_current)
    print('1')
  except:
    df_fin__= df_current
    print('2')

        Token          Tag
0        NÃºm            O
1           .            O
2         122            O
3           (            O
4           4            O
...       ...          ...
3614        ,            O
3615    calle  B-loc_direc
3616       de  I-loc_direc
3617  AlcalÃ¡  I-loc_direc
3618        .            O

[3619 rows x 2 columns]
2
           Token Tag
0           NÃºm   O
1              .   O
2             60   O
3            237   O
4         DIARIO   O
...          ...  ..
2189           .   O
2190         CON   O
2191  PRIVILEGIO   O
2192        REAL   O
2193           .   O

[2194 rows x 2 columns]
1
        Token          Tag
0        NÃºm            O
1           .            O
2         182            O
3           .            O
4           (            O
...       ...          ...
4716        ,            O
4717    calle  B-loc_direc
4718       de  I-loc_direc
4719  AlcalÃ¡  I-loc_direc
4720        .            O

[4721 rows x 2 columns]
1


In [14]:
train = df_fin__
val = pd.read_csv('/content/drive/MyDrive/Colab-Notebooks/TFM-HIPE/' + str(val), sep='\t', names=columns, encoding='latin1')
test = pd.read_csv('/content/drive/MyDrive/Colab-Notebooks/TFM-HIPE/' + str(test), sep='\t', names=columns, encoding='latin1')

In [15]:
train

,Token,Tag
0,NÃºm,O
1,.,O
2,122,O
3,(,O
4,4,O
...,...,...
4716,",",O
4717,calle,B-loc_direc
4718,de,I-loc_direc
4719,AlcalÃ¡,I-loc_direc


In [16]:
# solo si es necesario
def attempt_decode(x):
    try:
        return x.encode('latin-1').decode('utf-8')
    except UnicodeDecodeError:
       return x
       # f'Unable to decode: %s' %

train['Token'] = train['Token'].apply(attempt_decode)
val['Token'] = val['Token'].apply(attempt_decode)
test['Token'] = test['Token'].apply(attempt_decode)

In [17]:
test

,Token,Tag
0,Núm,O
1,.,O
2,335,O
3,1395,O
4,DIARIO,O
...,...,...
2290,SEÑORES,I-establec
2291,GARCIA,I-establec
2292,Y,I-establec
2293,COMPAÑIA,I-establec


In [18]:
import spacy
import es_core_news_sm
nlp = spacy.load('es_core_news_sm')
'''
text = 'Hola, Sra. Ramírez. He aquí su modelo.'
sentences = [i for i in nlp(text).sents]
sentences[0]
'''

"\ntext = 'Hola, Sra. Ramírez. He aquí su modelo.'\nsentences = [i for i in nlp(text).sents]\nsentences[0]\n"

In [19]:
def parse_clara(dataframe):
    
    """
    
    # conversión para capitel: # https://huggingface.co/PlanTL-GOB-ES/roberta-base-bne-capitel-ner/blob/main/config.json 

    dataframe = dataframe.replace({'B-pers_senores': 'B_PER'}, regex=True) # es importante poner B-pers_senores antes que B-pers, ya que si no, cambia B-pers_senores por B_PER_senores
    dataframe = dataframe.replace({'I-pers_senores': 'I_PER'}, regex=True)
    dataframe = dataframe.replace({'B-pers': 'B_PER'}, regex=True)
    dataframe = dataframe.replace({'I-pers': 'I_PER'}, regex=True)

    dataframe = dataframe.replace({'B-loc_direc': 'B_LOC'}, regex=True)
    dataframe = dataframe.replace({'I-loc_direc': 'I_LOC'}, regex=True)
    dataframe = dataframe.replace({'B-loc_hosp': 'B_LOC'}, regex=True)
    dataframe = dataframe.replace({'I-loc_hosp': 'I_LOC'}, regex=True)
    dataframe = dataframe.replace({'B-loc_relig': 'B_LOC'}, regex=True) # org?
    dataframe = dataframe.replace({'I-loc_relig': 'I_LOC'}, regex=True)
    dataframe = dataframe.replace({'B-loc_cole': 'B_LOC'}, regex=True) # org?
    dataframe = dataframe.replace({'I-loc_cole': 'I_LOC'}, regex=True)
    dataframe = dataframe.replace({'B-loc_fuente': 'B_MISC'}, regex=True)
    dataframe = dataframe.replace({'I-loc_fuente': 'I_MISC'}, regex=True)
    dataframe = dataframe.replace({'B-loc': 'B_LOC'}, regex=True)
    dataframe = dataframe.replace({'I-loc': 'I_LOC'}, regex=True)
    
    dataframe = dataframe.replace({'B-establec': 'B_ORG'}, regex=True) # loc?
    dataframe = dataframe.replace({'I-establec': 'I_ORG'}, regex=True)
    dataframe = dataframe.replace({'B-prof': 'B_OTH'}, regex=True) # o pers??
    dataframe = dataframe.replace({'I-prof': 'I_OTH'}, regex=True)
    dataframe = dataframe.replace({'B-dec_ador': 'B_OTH'}, regex=True)
    dataframe = dataframe.replace({'I-dec_ador': 'I_OTH'}, regex=True)
    dataframe = dataframe.replace({'B-dec_mobil': 'B_OTH'}, regex=True)
    dataframe = dataframe.replace({'I-dec_mobil': 'I_OTH'}, regex=True)
    dataframe = dataframe.replace({'B-venta': 'B_OTH'}, regex=True)
    dataframe = dataframe.replace({'I-venta': 'I_OTH'}, regex=True)
    dataframe = dataframe.replace({'B-perdida-hallazgo': 'B_OTH'}, regex=True)
    dataframe = dataframe.replace({'I-perdida-hallazgo': 'I_OTH'}, regex=True)

    for i in range(dataframe.shape[0]-1): # cambiar de I- a E- y de B- a S-
    # cambiar de B a S: si empieza con B y la siguiente NO empieza por I
      if dataframe.iloc[i]['Tag'].startswith('B') & (not dataframe.iloc[i+1]['Tag'].startswith('I')):
         dataframe.iloc[i]['Tag'] = dataframe.iloc[i]['Tag'].replace('B-', 'S-')
      # cambiar de I a E: si empieza por I y la siguiente no empieza por I 
      if dataframe.iloc[i]['Tag'].startswith('I') & (not dataframe.iloc[i+1]['Tag'].startswith('I')):
         dataframe.iloc[i]['Tag'] = dataframe.iloc[i]['Tag'].replace('I-', 'E-')
    

    # conversión para hipe

    dataframe = dataframe.replace({'B-pers_senores': 'B-pers'}, regex=True) # es importante poner B-pers_senores antes que B-pers, ya que si no, cambia B-pers_senores por B_PER_senores
    dataframe = dataframe.replace({'I-pers_senores': 'I-pers'}, regex=True)
    #dataframe = dataframe.replace({'B-pers': 'B-pers'}, regex=True)
    #dataframe = dataframe.replace({'I-pers': 'I-pers'}, regex=True)

    dataframe = dataframe.replace({'B-loc_direc': 'B-loc'}, regex=True)
    dataframe = dataframe.replace({'I-loc_direc': 'I-loc'}, regex=True)
    dataframe = dataframe.replace({'B-loc_hosp': 'B-loc'}, regex=True)
    dataframe = dataframe.replace({'I-loc_hosp': 'I-loc'}, regex=True)
    dataframe = dataframe.replace({'B-loc_relig': 'B-loc'}, regex=True) # org?
    dataframe = dataframe.replace({'I-loc_relig': 'I-loc'}, regex=True)
    dataframe = dataframe.replace({'B-loc_cole': 'B-loc'}, regex=True) # org?
    dataframe = dataframe.replace({'I-loc_cole': 'I-loc'}, regex=True)
    dataframe = dataframe.replace({'B-loc_fuente': 'O'}, regex=True)
    dataframe = dataframe.replace({'I-loc_fuente': 'O'}, regex=True)
    #dataframe = dataframe.replace({'B-loc': 'B-loc'}, regex=True)
    #dataframe = dataframe.replace({'I-loc': 'I-loc'}, regex=True)
    
    dataframe = dataframe.replace({'B-establec': 'B-org'}, regex=True) # loc?
    dataframe = dataframe.replace({'I-establec': 'I-org'}, regex=True)
    dataframe = dataframe.replace({'B-prof': 'O'}, regex=True) # o pers??
    dataframe = dataframe.replace({'I-prof': 'O'}, regex=True)
    dataframe = dataframe.replace({'B-dec_ador': 'O'}, regex=True)
    dataframe = dataframe.replace({'I-dec_ador': 'O'}, regex=True)
    dataframe = dataframe.replace({'B-dec_mobil': 'O'}, regex=True)
    dataframe = dataframe.replace({'I-dec_mobil': 'O'}, regex=True)
    dataframe = dataframe.replace({'B-venta': 'O'}, regex=True)
    dataframe = dataframe.replace({'I-venta': 'O'}, regex=True)
    dataframe = dataframe.replace({'B-perdida-hallazgo': 'O'}, regex=True)
    dataframe = dataframe.replace({'I-perdida-hallazgo': 'O'}, regex=True)
    
    """

    all_tokens = []
    all_tags = []

    for row in dataframe.itertuples():
      all_tokens.append(row.Token)
      all_tags.append(row.Tag)

    dataset_tokens_str = ' '.join(all_tokens)
    sentence_tokens = [str(i).split() for i in nlp(dataset_tokens_str).sents]

    sentence_tags = []
    counter = 0
    for sentence in sentence_tokens:
      length = len(sentence)
      sentence_tags.append(all_tags[counter:counter+length])
      counter = counter+length


    return sentence_tokens, sentence_tags

In [20]:
train_sents, train_tags = parse_clara(train)
val_sents, val_tags = parse_clara(val)
test_sents, test_tags = parse_clara(test)

In [21]:
print(f'numero de frases en el train: {len(train_sents)},  {len(train_tags)}')
print(f'numero de frases en el val: {len(val_sents)},  {len(val_tags)}')
print(f'numero de frases en el test: {len(test_sents)},  {len(test_tags)}')

numero de frases en el train: 698,  698
numero de frases en el val: 120,  120
numero de frases en el test: 111,  111


In [22]:
# ver cuánto mide la frase más larga
max_len = 50
i = 0
lista = []
for sentence in train: # en train hay 6 frases que miden más de 50 caracteres; en val hay 4 y en test hay 5
    if len(sentence) > max_len:
        lista.append( (i, len(sentence)) )
        max_len = len(sentence)
    i += 1
max_len, lista # lista con las posiciones de las frases que superan la longitud máxima 
               # en el test_texts del francés, la frase 64 mide 1158 tokens porque faltan las etiquetas de EndOfSentence
               # en el train_texts del francés, la frase 272 mide 529, la frase 319 mide 752 y la 3877 mide 2069 # esto era antes de solucionar las frases que empezaban por I-

(50, [])

In [23]:
sum_tokens = len([token for sentence in train_sents for token in sentence])
#sum_tags = len([token for sentence in train_tags for token in sentence ])
sum_tokens #, sum_tags

10534

In [24]:
# Importante: hay que guardar estos diccionarios junto con el modelo, ya que si se carga el modelo y se da otro diccionario de conversión se va a la porra todo
flat_tags = [tag for sublist in train_tags for tag in sublist]
unique_tags = sorted(list(set(flat_tags)))
#tag2id = {tag:id for id, tag in enumerate(unique_tags)}
#id2tag = {id:tag for tag, id in tag2id.items()}
#class_names = sorted(list(id2tag.values()))
class_names = unique_tags
class_names

['B-dec_ador',
 'B-establec',
 'B-loc',
 'B-loc_direc',
 'B-loc_hosp',
 'B-loc_relig',
 'B-perdida-hallazgo',
 'B-pers',
 'B-pers_senores',
 'B-prof',
 'B-venta',
 'I-establec',
 'I-loc',
 'I-loc_direc',
 'I-loc_hosp',
 'I-loc_relig',
 'I-perdida-hallazgo',
 'I-pers',
 'I-pers_senores',
 'I-prof',
 'I-venta',
 'O']

In [25]:
#class_names = ['O', 'B-pers', 'I-pers', 'B-org', 'I-org', 'B-loc', 'I-loc', 'B-prod', 'I-prod', 'B-time', 'I-time'] 

#class_names = ["I-PER", "I-LOC", "B-ORG", "I-ORG", "I-MISC", "B-MISC", "O", "B-PER", "B-LOC"]
#class_names = ["B_OTH", "I_OTH", "E_OTH", "S_ORG", "S_OTH", "B_PER", "E_PER", "B_ORG", "E_ORG", "S_LOC", "S_PER", "B_LOC", "E_LOC", "I_PER", "I_ORG", "I_LOC", "O"]

class_names = ['O', 'B-loc', 'I-loc', 'B-loc_direc', 'I-loc_direc', 'B-loc_hosp', 'I-loc_hosp', 'B-loc_relig', 'I-loc_relig', 'B-loc_cole', 'I-loc_cole', 'B-pers', 'I-pers', \
               'B-pers_senores', 'I-pers_senores', 'B-prof', 'I-prof', 'B-establec', 'I-establec', 'B-dec_ador', 'I-dec_ador', 'B-perdida-hallazgo', 'I-perdida-hallazgo', 'B-venta', 'I-venta']

In [26]:
train_dataset_dict = {"tokens": train_sents, "ner_tags": train_tags} 
val_dataset_dict = {"tokens": val_sents, "ner_tags": val_tags} 
test_dataset_dict = {"tokens": test_sents, "ner_tags": test_tags} 

features = Features({'tokens': Sequence(Value("string")), 'ner_tags': Sequence(ClassLabel(names=class_names))})

train_dataset = Dataset.from_dict(train_dataset_dict, features=features)
val_dataset = Dataset.from_dict(val_dataset_dict, features=features)
test_dataset = Dataset.from_dict(test_dataset_dict, features=features)

In [27]:
ner_dataset = DatasetDict()
ner_dataset["train"] = train_dataset
ner_dataset["val"] = val_dataset
ner_dataset["test"] = test_dataset

In [28]:
ner_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 698
    })
    val: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 120
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 111
    })
})

In [29]:
tags = ner_dataset["train"].features["ner_tags"].feature
# hide_output
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}

In [30]:
ner_dataset = ner_dataset.map(create_tag_names)

  0%|          | 0/698 [00:00<?, ?ex/s]

  0%|          | 0/120 [00:00<?, ?ex/s]

  0%|          | 0/111 [00:00<?, ?ex/s]

In [67]:
example = ner_dataset["train"][30]
example

{'tokens': ['263', 'D.', 'Miguel', 'Gandulla', '.'],
 'ner_tags': [0, 11, 12, 12, 0],
 'ner_tags_str': ['O', 'B-pers', 'I-pers', 'I-pers', 'O']}

In [68]:
pd.DataFrame([example['tokens'], example['ner_tags_str']], ['Tokens', 'Tags'])

,0,1,2,3,4
Tokens,263,D.,Miguel,Gandulla,.
Tags,O,B-pers,I-pers,I-pers,O


In [69]:
ner_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 698
    })
    val: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 120
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 111
    })
})

In [25]:
split2freqs = defaultdict(Counter)
for split, dataset in ner_dataset.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient="index")

,loc,pers_senores,loc_relig,pers,establec,prof,loc_direc,loc_hosp,perdida,venta,dec_ador,loc_cole
train,51,29,21,322,46,63,76,2.0,12.0,6,4.0,NaN
val,6,7,1,17,6,5,13,NaN,2.0,14,NaN,NaN
test,21,13,6,8,29,21,23,NaN,NaN,12,NaN,1.0


In [31]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

In [32]:
index2tag

{0: 'O',
 1: 'B-loc',
 2: 'I-loc',
 3: 'B-loc_direc',
 4: 'I-loc_direc',
 5: 'B-loc_hosp',
 6: 'I-loc_hosp',
 7: 'B-loc_relig',
 8: 'I-loc_relig',
 9: 'B-loc_cole',
 10: 'I-loc_cole',
 11: 'B-pers',
 12: 'I-pers',
 13: 'B-pers_senores',
 14: 'I-pers_senores',
 15: 'B-prof',
 16: 'I-prof',
 17: 'B-establec',
 18: 'I-establec',
 19: 'B-dec_ador',
 20: 'I-dec_ador',
 21: 'B-perdida-hallazgo',
 22: 'I-perdida-hallazgo',
 23: 'B-venta',
 24: 'I-venta'}

In [33]:
from transformers import AutoTokenizer

bert_model_name = "bert-base-cased"
model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(model_name)

In [74]:
text = "Jack Sparrow aime New York !"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

In [ ]:
#hide_input
df = pd.DataFrame([bert_tokens, xlmr_tokens], index=["BERT", "XLM-R"])
df

,0,1,2,3,4,5,6,7,8,9
BERT,[CLS],Jack,Spa,##rrow,aim,##e,New,York,!,[SEP]
XLM-R,<s>,▁Jack,▁Spar,row,▁aime,▁New,▁York,▁!,</s>,None


In [ ]:
"".join(xlmr_tokens).replace(u"\u2581", " ")

'<s> Jack Sparrow aime New York !</s>'

In [ ]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        # Load model body
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        # Set up token classification head
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # Load and initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, 
                labels=None, **kwargs):
        # Use model body to get encoder representations
        outputs = self.roberta(input_ids, attention_mask=attention_mask,
                               token_type_ids=token_type_ids, **kwargs)
        # Apply classifier to encoder representation
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        # Calculate losses
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        # Return model output object
        return TokenClassifierOutput(loss=loss, logits=logits, 
                                     hidden_states=outputs.hidden_states, 
                                     attentions=outputs.attentions)

In [34]:
# hide_output
from transformers import AutoConfig

xlmr_config = AutoConfig.from_pretrained(model_name, 
                                         num_labels=tags.num_classes,
                                         id2label=index2tag, label2id=tag2index)

In [35]:
# hide_output
import torch
from transformers import AutoModelForTokenClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xlmr_model = (AutoModelForTokenClassification #XLMRobertaForTokenClassification
              .from_pretrained(model_name, config=xlmr_config)
              .to(device))

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [ ]:
# hide_output
input_ids = xlmr_tokenizer.encode(text, return_tensors="pt")
pd.DataFrame([xlmr_tokens, input_ids[0].numpy()], index=["Tokens", "Input IDs"])

,0,1,2,3,4,5,6,7,8
Tokens,<s>,▁Jack,▁Spar,row,▁aime,▁New,▁York,▁!,</s>
Input IDs,0,21763,37456,15555,91022,2356,5753,711,2


In [ ]:
outputs = xlmr_model(input_ids.to(device)).logits
predictions = torch.argmax(outputs, dim=-1)
print(f"Number of tokens in sequence: {len(xlmr_tokens)}")
print(f"Shape of outputs: {outputs.shape}")

Number of tokens in sequence: 9
Shape of outputs: torch.Size([1, 9, 11])


In [ ]:
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens, preds], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8
Tokens,<s>,▁Jack,▁Spar,row,▁aime,▁New,▁York,▁!,</s>
Tags,B-loc,B-loc,B-loc,B-loc,B-loc,B-loc,B-loc,B-loc,B-loc


In [ ]:
def tag_text(text, tags, model, tokenizer):
    # Get tokens with special characters
    tokens = tokenizer(text).tokens()
    # Encode the sequence into IDs
    input_ids = xlmr_tokenizer(text, return_tensors="pt").input_ids.to(device)
    # Get predictions as distribution over 7 possible classes
    outputs = model(input_ids)[0]
    # Take argmax to get most likely class per token
    predictions = torch.argmax(outputs, dim=2)
    # Convert to DataFrame
    preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])

In [ ]:
words, labels = example["tokens"], example["ner_tags"]

In [ ]:
tokenized_input = xlmr_tokenizer(example["tokens"], is_split_into_words=True)
tokens = xlmr_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [ ]:
#hide_output
pd.DataFrame([tokens], index=["Tokens"])

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124
Tokens,<s>,▁digo,▁que,▁Bert,in,▁pone,▁un,▁punto,▁redo,ndo,...,pu,es,▁de,▁un,▁año,▁de,▁práctica,▁,.,</s>


In [ ]:
# hide_output
word_ids = tokenized_input.word_ids()
pd.DataFrame([tokens, word_ids], index=["Tokens", "Word IDs"])

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124
Tokens,<s>,▁digo,▁que,▁Bert,in,▁pone,▁un,▁punto,▁redo,ndo,...,pu,es,▁de,▁un,▁año,▁de,▁práctica,▁,.,</s>
Word IDs,None,0,1,2,2,3,4,5,6,6,...,85,85,86,87,88,89,90,91,91,None


In [ ]:
#hide_output
previous_word_idx = None
label_ids = []

for word_idx in word_ids:
    if word_idx is None or word_idx == previous_word_idx:
        label_ids.append(-100)
    elif word_idx != previous_word_idx:
        label_ids.append(labels[word_idx])
    previous_word_idx = word_idx
    
labels = [index2tag[l] if l != -100 else "IGN" for l in label_ids]
index = ["Tokens", "Word IDs", "Label IDs", "Labels"]

pd.DataFrame([tokens, word_ids, label_ids, labels], index=index)

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124
Tokens,<s>,▁digo,▁que,▁Bert,in,▁pone,▁un,▁punto,▁redo,ndo,...,pu,es,▁de,▁un,▁año,▁de,▁práctica,▁,.,</s>
Word IDs,None,0,1,2,2,3,4,5,6,6,...,85,85,86,87,88,89,90,91,91,None
Label IDs,-100,0,0,1,-100,0,0,0,0,-100,...,-100,-100,0,0,0,0,0,0,-100,-100
Labels,IGN,O,O,B-pers,IGN,O,O,O,O,IGN,...,IGN,IGN,O,O,O,O,O,O,IGN,IGN


In [36]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(examples["tokens"], truncation=True, 
                                      is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [37]:
def encode_dataset(corpus):
    return corpus.map(tokenize_and_align_labels, batched=True, 
                      remove_columns=['ner_tags', 'tokens'])

In [38]:
clara_encoded = encode_dataset(ner_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Cargar HIPE

In [ ]:
# german
traindata_de = "https://raw.githubusercontent.com/hipe-eval/HIPE-2022-data/main/data/v2.1/hipe2020/de/HIPE-2022-v2.1-hipe2020-train-de.tsv"
valdata_de = "https://raw.githubusercontent.com/hipe-eval/HIPE-2022-data/main/data/v2.1/hipe2020/de/HIPE-2022-v2.1-hipe2020-dev-de.tsv"
testdata_de = "https://raw.githubusercontent.com/hipe-eval/HIPE-2022-data/main/data/v2.1/hipe2020/de/HIPE-2022-v2.1-hipe2020-test_ELmasked-de.tsv"

# french
traindata_fr = "https://raw.githubusercontent.com/hipe-eval/HIPE-2022-data/main/data/v2.1/hipe2020/fr/HIPE-2022-v2.1-hipe2020-train-fr.tsv"
valdata_fr = "https://raw.githubusercontent.com/hipe-eval/HIPE-2022-data/main/data/v2.1/hipe2020/fr/HIPE-2022-v2.1-hipe2020-dev-fr.tsv"
testdata_fr = "https://raw.githubusercontent.com/hipe-eval/HIPE-2022-data/main/data/v2.1/hipe2020/fr/HIPE-2022-v2.1-hipe2020-test-fr.tsv"

# english
valdata_en = "https://raw.githubusercontent.com/hipe-eval/HIPE-2022-data/main/data/v2.1/hipe2020/en/HIPE-2022-v2.1-hipe2020-dev-en.tsv"
testdata_en = "https://raw.githubusercontent.com/hipe-eval/HIPE-2022-data/main/data/v2.1/hipe2020/en/HIPE-2022-v2.1-hipe2020-test-en.tsv"

def parse_dataset(dataset):
    """
    Args:
        df (pandas dataframe): Original dataset in pandas format
    """
    #dataset.dropna(how="all", inplace=True)

    dataset = dataset[['TOKEN', 'NE-COARSE-LIT', 'MISC']]

    exclude = [getattr(i, 'Index') for i in dataset.itertuples() if (getattr(i, 'TOKEN')).startswith('#')]

    dataset = dataset[~dataset.index.isin(exclude)]

    # Renombramos la segunda columna para que el siguiente paso no dé problemas
    dataset.rename(columns = {'TOKEN' : 'TOKEN', 'NE-COARSE-LIT' : 'NE', 'MISC' : 'MISC'}, inplace=True)


    '''
    ### Esto es para ajustar las etiquetas a las de https://huggingface.co/Davlan/xlm-roberta-base-ner-hrl
    dataset = dataset.replace({'B-prod': 'O'}, regex=True)
    dataset = dataset.replace({'I-prod': 'O'}, regex=True)
    dataset = dataset.replace({'B-time': 'B-DATE'}, regex=True)
    dataset = dataset.replace({'I-time': 'I-DATE'}, regex=True)
    dataset = dataset.replace({'B-pers': 'B-PER'}, regex=True)
    dataset = dataset.replace({'I-pers': 'I-PER'}, regex=True)
    dataset = dataset.replace({'B-org': 'B-ORG'}, regex=True)
    dataset = dataset.replace({'I-org': 'I-ORG'}, regex=True)
    dataset = dataset.replace({'B-loc': 'B-LOC'}, regex=True)
    dataset = dataset.replace({'I-loc': 'I-LOC'}, regex=True)
    '''


    all_sentences = []
    all_tags = []

    sentence_tokens = []
    ner_tags = []

    #i = 0
    for row in dataset.itertuples():
        sentence_tokens.append(row.TOKEN)
        ner_tags.append(row.NE)
        #i += 1
        if (row.MISC == 'EndOfSentence' or row.MISC == 'EndOfLine|EndOfSentence') and row.NE == 'O': # or row.TOKEN == '.': esto hace que se separen entidades como M. Hagenbeck. Pero las etiquetas 
            all_sentences.append(sentence_tokens)                                                    # EndOfSentence también están mal y a veces cortan las entidades!! Habria que usar un sentencesplitter
            all_tags.append(ner_tags)                                                                # or len(sentence_tokens) > 214: esto tampoco mejora los resultados
                                                                                                    # row.NE == 'O' evita que una frase empiece con etiqueta I- ya que se asegura de que el endofsentence no está
            sentence_tokens = []                                                                    # en medio de una entidad
            ner_tags = []



    return all_sentences, all_tags

# alemán
traindf_de = pd.read_csv(traindata_de, sep='\t', quoting=3, encoding='utf-8', comment="#", skip_blank_lines=True).dropna() 
valdf_de = pd.read_csv(valdata_de, sep='\t', quoting=3, encoding='utf-8', comment="#", skip_blank_lines=True).dropna()
testdf_de = pd.read_csv(testdata_de, sep='\t', quoting=csv.QUOTE_NONE, encoding='utf-8', comment="#", skip_blank_lines=True).dropna()

#french
traindf_fr = pd.read_csv(traindata_fr, sep='\t', quoting=3, encoding='utf-8', comment="#", skip_blank_lines=True).dropna() 
valdf_fr = pd.read_csv(valdata_fr, sep='\t', quoting=3, encoding='utf-8', comment="#", skip_blank_lines=True).dropna()
testdf_fr = pd.read_csv(testdata_fr, sep='\t', quoting=3, encoding='utf-8', comment="#", skip_blank_lines=True).dropna()

# inglés
valdf_en = pd.read_csv(valdata_en, sep='\t', quoting=3, encoding='utf-8', comment="#", skip_blank_lines=True).dropna()
testdf_en = pd.read_csv(testdata_en, sep='\t', quoting=3, encoding='utf-8', comment="#", skip_blank_lines=True).dropna()

In [ ]:
# unir train fr y test en y val en + val fr y val en
traindf_fr_en = traindf_fr.append(testdf_en)
valdf_fr_en = valdf_fr.append(valdf_en)

In [ ]:
#train_texts_fr, train_tags_fr = parse_dataset(traindf_fr)
#train_texts_de, train_tags_de = parse_dataset(traindf_de)

#val_texts_fr, val_tags_fr =  parse_dataset(valdf_fr)
#val_texts_de, val_tags_de =  parse_dataset(valdf_de)
#val_texts_en, val_tags_en =  parse_dataset(valdf_en)

train_texts_fr_en, train_tags_fr_en = parse_dataset(traindf_fr_en)
val_texts_fr_en, val_tags_fr_en = parse_dataset(valdf_fr_en)

test_texts_en, test_tags_en =  parse_dataset(testdf_en)
test_texts_de, test_tags_de =  parse_dataset(testdf_de)
test_texts_fr, test_tags_fr =  parse_dataset(testdf_fr)

In [ ]:
print(f'train length: {len(train_texts_fr_en)},  {len(train_tags_fr_en)}, validation/development length: {len(val_texts_fr_en)}, {len(val_tags_fr_en)}') #, test length: {len(test_texts_fr)}, {len(test_tags_fr)}')
#print(f'train deutsch length: {len(train_texts_de)},  {len(train_tags_de)}, validation/development length: {len(val_texts_de)}, {len(val_tags_de)}), test length: {len(test_texts_de)}, {len(test_tags_de)}')
#print(f'english validation/development length: {len(val_texts_en)}, {len(val_tags_en)}), test length: {len(test_texts_en)}, {len(test_tags_en)}')

train length: 5833,  5833, validation/development length: 2124, 2124


In [ ]:
# ver cuánto mide la frase más larga
max_len = 50
i = 0
lista = []
for sentence in train_texts_fr_en: # en train hay 6 frases que miden más de 50 caracteres; en val hay 4 y en test hay 5
    if len(sentence) > max_len:
        lista.append( (i, len(sentence)) )
        max_len = len(sentence)
    i += 1
print(max_len, lista) # lista con las posiciones de las frases que superan la longitud máxima 
               # en el test_texts del francés, la frase 64 mide 1158 tokens porque faltan las etiquetas de EndOfSentence
               # en el train_texts del francés, la frase 272 mide 529, la frase 319 mide 752 y la 3877 mide 2069 # esto era antes de solucionar las frases que empezaban por I-



2178 [(7, 63), (13, 89), (32, 170), (266, 529), (312, 752), (3719, 2178)]


In [ ]:
sum_tokens_fr = len([token for sentence in train_texts_fr for token in sentence ])
sum_tokens_de = len([token for sentence in train_texts_de for token in sentence ])
sum_tokens_en = len([token for sentence in test_texts_en for token in sentence ])
sum_tokens_fr, sum_tokens_de, sum_tokens_en

(166217, 86444, 16634)

In [ ]:
# ya las hemos definido en CLARA

# Importante: hay que guardar estos diccionarios junto con el modelo, ya que si se carga el modelo y se da otro diccionario de conversión se va a la porra todo
#unique_tags = sorted(list(set(traindf['NE-COARSE-LIT'])))
#tag2id = {tag:id for id, tag in enumerate(unique_tags)}
#id2tag = {id:tag for tag, id in tag2id.items()}
#class_names = sorted(list(id2tag.values()))
#class_names = unique_tags
#print(class_names)
#class_names = ['O', 'B-pers', 'I-pers', 'B-org', 'I-org', 'B-loc', 'I-loc', 'B-prod', 'I-prod', 'B-time', 'I-time',] # mismo orden que panx aunque nombres distintos, cuidado
#class_names = ['O', 'B-DATE', 'I-DATE', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'] # 'B-prod', 'I-prod', ] # orden y etiquetas de https://huggingface.co/Davlan/xlm-roberta-base-ner-hrl/blob/main/config.json 


In [ ]:
# fr + en
train_dataset_dict_fr_en = {"tokens":train_texts_fr_en, "ner_tags":train_tags_fr_en}
val_dataset_dict_fr_en = {"tokens":val_texts_fr_en, "ner_tags":val_tags_fr_en}

# fr
#test_dataset_dict_fr = {"tokens":test_texts_fr, "ner_tags":test_tags_fr} 
features = Features({'tokens': Sequence(Value("string")), 'ner_tags': Sequence(ClassLabel(names=class_names))})



#fr y en
ner_dataset_train = Dataset.from_dict(train_dataset_dict_fr_en, features=features)
ner_dataset_val = Dataset.from_dict(val_dataset_dict_fr_en, features=features)

#ner_dataset_test_fr = Dataset.from_dict(test_dataset_dict_fr, features=features)


'''
#en
ner_dataset_val_en = Dataset.from_dict(val_dataset_dict_en, features=features)
ner_dataset_test_en = Dataset.from_dict(test_dataset_dict_en, features=features)
ner_dataset_fr = DatasetDict()
'''

ner_dataset_hipe = DatasetDict()
ner_dataset_hipe["train"] = ner_dataset_train
ner_dataset_hipe["validation"] = ner_dataset_val
#ner_dataset["test"] = ner_dataset_test_fr

tags = ner_dataset_hipe["train"].features["ner_tags"].feature

def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}

ner_dataset_hipe = ner_dataset_hipe.map(create_tag_names)
#ner_dataset_de = ner_dataset_de.map(create_tag_names)
#ner_dataset_en = ner_dataset_en.map(create_tag_names)

  0%|          | 0/5833 [00:00<?, ?ex/s]

  0%|          | 0/2124 [00:00<?, ?ex/s]

In [ ]:
ner_dataset_hipe

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 5833
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 2124
    })
})

In [ ]:
split2freqs = defaultdict(Counter)
for split, dataset in ner_dataset_hipe.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient="index")

,time,org,loc,pers,prod
train,293,912,3270,2681,219
validation,97,277,1158,1081,82


In [ ]:
#clara

'''
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

from transformers import AutoTokenizer


model_name = "xlm-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(examples["tokens"], truncation=True, 
                                      is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

def encode_dataset(corpus):
    return corpus.map(tokenize_and_align_labels, batched=True, 
                      remove_columns=['ner_tags', 'tokens'])
'''
hipe_encoded = encode_dataset(ner_dataset_hipe)


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

# función para integrar las métricas de seqeval durante el entrenamiento
To integrate these metrics during training, we need a function that can take the outputs of the model and convert them into the lists that seqeval expects. The following does the trick by ensuring we ignore the label IDs associated with subsequent subwords:

In [39]:
import numpy as np

def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore label IDs = -100
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

In [40]:
from seqeval.metrics import f1_score, precision_score, recall_score, accuracy_score #, classification_report

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions, 
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred), "precision": precision_score(y_true, y_pred), "recall": recall_score(y_true, y_pred)} #, "report": classification_report(y_true, y_pred)}

# finetuning

In [42]:
# hide_output
from transformers import TrainingArguments

num_epochs = 10
batch_size = 12 
logging_steps = len(clara_encoded["train"]) // batch_size
model_name_output = "xlm-roberta-clara" #f"./results"
training_args = TrainingArguments(
    output_dir=model_name_output,
    log_level="error",
    num_train_epochs=num_epochs, 
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch", 
    save_steps=1e6,
    weight_decay=0.01,
    disable_tqdm=False, 
    logging_steps=logging_steps,
    push_to_hub=True)

### tell the trainer how to compute metrics

In [43]:
from seqeval.metrics import f1_score, precision_score, recall_score, accuracy_score #, classification_report

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions, 
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred), "precision": precision_score(y_true, y_pred), "recall": recall_score(y_true, y_pred)} #, "report": classification_report(y_true, y_pred)}

data collator so we can pad each input sequence to the largest sequence length in a batch. nlpt_pin01 Transformers provides a dedicated data collator for token classification that will pad the labels along with the inputs:

In [44]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [45]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [46]:
from transformers import AutoModelForTokenClassification

def model_init():
    return (AutoModelForTokenClassification # XLMRobertaForTokenClassification
            .from_pretrained(model_name, config=xlmr_config)
            .to(device))

In [47]:
#hide
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [48]:
# hide_output
from transformers import Trainer

trainer = Trainer(model_init=model_init,
                  args=training_args, 
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  train_dataset=clara_encoded["train"],
                  eval_dataset=clara_encoded["val"], 
                  tokenizer=xlmr_tokenizer)

/content/xlm-roberta-clara is already a clone of https://huggingface.co/evs/xlm-roberta-clara. Make sure you pull the latest changes with `repo.git_pull()`.


In [49]:
trainer.train()
trainer.push_to_hub(commit_message="Training completed!")

Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.824800,0.340513,0.120000,0.222222,0.082192
2,0.365200,0.266752,0.377049,0.469388,0.315068
3,0.238300,0.276359,0.365591,0.300885,0.465753
4,0.172200,0.257695,0.342857,0.294118,0.410959
5,0.117900,0.222645,0.395480,0.336538,0.479452
6,0.080600,0.235267,0.412500,0.379310,0.452055
7,0.059100,0.233013,0.409091,0.349515,0.493151
8,0.042900,0.286624,0.444444,0.387755,0.520548
9,0.024700,0.290075,0.425287,0.366337,0.506849
10,0.023700,0.302706,0.431818,0.368932,0.520548


Upload file pytorch_model.bin:   0%|          | 3.33k/1.03G [00:00<?, ?B/s]

Upload file runs/Sep16_09-18-07_787d203ce7dd/1663319917.3963888/events.out.tfevents.1663319917.787d203ce7dd.17…

Upload file training_args.bin: 100%|##########| 2.80k/2.80k [00:00<?, ?B/s]

Upload file runs/Sep16_09-18-07_787d203ce7dd/events.out.tfevents.1663319917.787d203ce7dd.1770.0:  33%|###2    …

...

Upload file runs/Sep16_08-58-40_787d203ce7dd/events.out.tfevents.1663318763.787d203ce7dd.70.3:  78%|#######8  …

Upload file runs/Sep16_08-58-40_787d203ce7dd/1663318763.9420822/events.out.tfevents.1663318763.787d203ce7dd.70…

Upload file runs/Sep16_08-36-12_787d203ce7dd/events.out.tfevents.1663318467.787d203ce7dd.70.2: 100%|##########…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/evs/xlm-roberta-clara
   61b9c45..b7d22f5  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/evs/xlm-roberta-clara
   61b9c45..b7d22f5  main -> main

To https://huggingface.co/evs/xlm-roberta-clara
   b7d22f5..d928232  main -> main

   b7d22f5..d928232  main -> main



'https://huggingface.co/evs/xlm-roberta-clara/commit/b7d22f57e091ed6f74b938e4ea8e0246e06ca6bf'

In [ ]:
# hide_input
df = pd.DataFrame(trainer.state.log_history)[['epoch','loss' ,'eval_loss', 'eval_f1']]
df = df.rename(columns={"epoch":"Epoch","loss": "Training Loss", "eval_loss": "Validation Loss", "eval_f1":"F1"})
df['Epoch'] = df["Epoch"].apply(lambda x: round(x))
df['Training Loss'] = df["Training Loss"].ffill()
df[['Validation Loss', 'F1']] = df[['Validation Loss', 'F1']].bfill().ffill()
df.drop_duplicates()

,Epoch,Training Loss,Validation Loss,F1
0,1,0.1866,0.122689,0.683440
2,2,0.0683,0.119382,0.724274
4,3,0.0356,0.117819,0.748770


In [ ]:
# hide_output
text = "Jack Sparrow aime New York !"
tag_text(text, tags, trainer.model, xlmr_tokenizer)

,0,1,2,3,4,5,6,7,8
Tokens,<s>,▁Jack,▁Spar,row,▁aime,▁New,▁York,▁!,</s>
Tags,O,B-pers,I-pers,I-pers,O,B-loc,I-loc,O,O


In [ ]:
text = "Juana Inés est née dans la propriété de son grand-père, près d'une petite communauté rurale du Mexique, San Miguel Nepantla."
tag_text(text, tags, trainer.model, xlmr_tokenizer)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
Tokens,<s>,▁Juan,a,▁In,és,▁est,▁,née,▁dans,▁la,...,▁Mex,ique,",",▁San,▁Miguel,▁Ne,pant,la,.,</s>
Tags,O,B-pers,I-pers,I-pers,I-pers,O,O,O,O,O,...,B-loc,I-pers,O,B-pers,I-pers,I-pers,I-pers,I-pers,O,O


In [50]:
trainer.evaluate(clara_encoded["test"])

{'eval_loss': 0.4391734004020691,
 'eval_f1': 0.45751633986928103,
 'eval_precision': 0.4093567251461988,
 'eval_recall': 0.5185185185185185,
 'eval_runtime': 0.9364,
 'eval_samples_per_second': 118.545,
 'eval_steps_per_second': 10.68,
 'epoch': 10.0}

In [ ]:
trainer.evaluate(hipe_fr_encoded["test"])

{'eval_loss': 0.09617626667022705,
 'eval_f1': 0.7792792792792793,
 'eval_precision': 0.7582968065122104,
 'eval_recall': 0.8014559894109861,
 'eval_runtime': 12.4393,
 'eval_samples_per_second': 109.893,
 'eval_steps_per_second': 9.164,
 'epoch': 3.0}

In [ ]:
outputs = trainer.predict(hipe_fr_encoded["test"])

In [ ]:
outputs.metrics

{'test_loss': 0.09617626667022705,
 'test_f1': 0.7792792792792793,
 'test_precision': 0.7582968065122104,
 'test_recall': 0.8014559894109861,
 'test_runtime': 12.5162,
 'test_samples_per_second': 109.219,
 'test_steps_per_second': 9.108}

# report clasificación con seqeval

In [ ]:
from seqeval.metrics import classification_report

y_true = [["O", "O", "O", "B-MISC", "I-MISC", "I-MISC", "O"],
          ["B-PER", "I-PER", "O"]]
y_pred = [["O", "O", "B-MISC", "I-MISC", "I-MISC", "I-MISC", "O"],
          ["B-PER", "I-PER", "O"]]
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

        MISC       0.00      0.00      0.00         1
         PER       1.00      1.00      1.00         1

   micro avg       0.50      0.50      0.50         2
   macro avg       0.50      0.50      0.50         2
weighted avg       0.50      0.50      0.50         2



In [ ]:
outputs

PredictionOutput(predictions=array([[[ 1.99516988e+00, -3.90334070e-01,  3.78968865e-01, ...,
         -6.48822188e-01, -5.87161005e-01,  6.44514710e-02],
        [ 2.85306835e+00,  3.53745246e+00, -4.56664830e-01, ...,
         -2.38065362e+00, -1.82088590e+00, -1.55468559e+00],
        [ 3.37422776e+00,  8.75054121e-01,  2.15368223e+00, ...,
         -1.96775568e+00, -1.96792960e+00, -9.30159628e-01],
        ...,
        [-1.00000000e+02, -1.00000000e+02, -1.00000000e+02, ...,
         -1.00000000e+02, -1.00000000e+02, -1.00000000e+02],
        [-1.00000000e+02, -1.00000000e+02, -1.00000000e+02, ...,
         -1.00000000e+02, -1.00000000e+02, -1.00000000e+02],
        [-1.00000000e+02, -1.00000000e+02, -1.00000000e+02, ...,
         -1.00000000e+02, -1.00000000e+02, -1.00000000e+02]],

       [[ 3.15746975e+00, -2.05106393e-01,  2.36785442e-01, ...,
         -8.33862722e-01, -5.65511882e-01, -4.90219928e-02],
        [ 1.05554943e+01, -6.40719354e-01, -9.68909144e-01, ...,
         

In [ ]:
y_pred, y_true = align_predictions(outputs.predictions,  outputs.label_ids)# predictions, label_ids)

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

In [ ]:
print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

         loc       0.83      0.86      0.84       782
         org       0.56      0.64      0.59       125
        pers       0.74      0.80      0.77       495
        prod       0.61      0.54      0.57        57
        time       0.54      0.60      0.57        52

   micro avg       0.76      0.80      0.78      1511
   macro avg       0.66      0.69      0.67      1511
weighted avg       0.76      0.80      0.78      1511



# Inferencia en test set cargando el modelo ALEMÁN + FRANCÉS desde huggingface

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained("evs/xlm-roberta-hipe2020-fr-de")

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

In [ ]:
import numpy as np

def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore label IDs = -100
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

In [ ]:
from seqeval.metrics import f1_score, precision_score, recall_score, accuracy_score #, classification_report

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions, 
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred), "precision": precision_score(y_true, y_pred), "recall": recall_score(y_true, y_pred)} #, "report": classification_report(y_true, y_pred)}

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [ ]:
test_args = TrainingArguments(
    output_dir = "./results",
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 8,   
    dataloader_drop_last = False    
)

# init trainer
trainer = Trainer(
              model = model, 
              data_collator=data_collator,
              args = test_args, 
              compute_metrics = compute_metrics)

In [ ]:
test_results = trainer.predict(clara_encoded["test"])
test_results.metrics

The following columns in the test set  don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: ner_tags_str.
***** Running Prediction *****
  Num examples = 928
  Batch size = 8


{'test_loss': 0.44906148314476013,
 'test_f1': 0.4616376531270148,
 'test_precision': 0.4349939246658566,
 'test_recall': 0.49175824175824173,
 'test_runtime': 5.3742,
 'test_samples_per_second': 172.676,
 'test_steps_per_second': 21.585}

# inferencia 2: comprobación de que es lo mismo entrenar en fr y evaluar en clara, que coger el modelo entrenado en fr desde huggingface y hacer inferencia en clara.

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained("evs/xlm-roberta-finetuned-hipe2020-fr-4")

https://huggingface.co/evs/xlm-roberta-finetuned-hipe2020-fr-4/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpwtyx_33j


Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

storing https://huggingface.co/evs/xlm-roberta-finetuned-hipe2020-fr-4/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/6dc5b3387cace010b52e23a330333111d604a97b28072a940d5a8e9fd0c15e5f.4f0e23d9a9e217f14d63d2b4d3c1749d755457cffb86c496c7a387c1cdcd28ad
creating metadata file for /root/.cache/huggingface/transformers/6dc5b3387cace010b52e23a330333111d604a97b28072a940d5a8e9fd0c15e5f.4f0e23d9a9e217f14d63d2b4d3c1749d755457cffb86c496c7a387c1cdcd28ad
loading configuration file https://huggingface.co/evs/xlm-roberta-finetuned-hipe2020-fr-4/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6dc5b3387cace010b52e23a330333111d604a97b28072a940d5a8e9fd0c15e5f.4f0e23d9a9e217f14d63d2b4d3c1749d755457cffb86c496c7a387c1cdcd28ad
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos

Downloading:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

storing https://huggingface.co/evs/xlm-roberta-finetuned-hipe2020-fr-4/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/88e24a831348aa19a896dba256b598b55b638b19887010c1bf17ecc968baa694.d39400a5480ca4c92e41a1c586773771f444ac767ff3e6ee3bce9a4a5ece5af1
creating metadata file for /root/.cache/huggingface/transformers/88e24a831348aa19a896dba256b598b55b638b19887010c1bf17ecc968baa694.d39400a5480ca4c92e41a1c586773771f444ac767ff3e6ee3bce9a4a5ece5af1
loading weights file https://huggingface.co/evs/xlm-roberta-finetuned-hipe2020-fr-4/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/88e24a831348aa19a896dba256b598b55b638b19887010c1bf17ecc968baa694.d39400a5480ca4c92e41a1c586773771f444ac767ff3e6ee3bce9a4a5ece5af1
All model checkpoint weights were used when initializing XLMRobertaForTokenClassification.

All the weights of XLMRobertaForTokenClassification were initialized from the model checkpoint at evs/xlm-roberta-finetuned-hipe2020-f

In [ ]:
import numpy as np

def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore label IDs = -100
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

In [ ]:
from seqeval.metrics import f1_score, precision_score, recall_score, accuracy_score #, classification_report

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions, 
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred), "precision": precision_score(y_true, y_pred), "recall": recall_score(y_true, y_pred)} #, "report": classification_report(y_true, y_pred)}

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [ ]:
test_args = TrainingArguments(
    output_dir = "./results",
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 8,   
    dataloader_drop_last = False    
)

# init trainer
trainer = Trainer(
              model = model, 
              data_collator=data_collator,
              args = test_args, 
              compute_metrics = compute_metrics)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
test_results = trainer.predict(clara_encoded["test"])
test_results.metrics

The following columns in the test set  don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: ner_tags_str.
***** Running Prediction *****
  Num examples = 928
  Batch size = 8


{'test_loss': 0.4072078466415405,
 'test_f1': 0.4759586122945831,
 'test_precision': 0.4273224043715847,
 'test_recall': 0.5370879120879121,
 'test_runtime': 5.9342,
 'test_samples_per_second': 156.381,
 'test_steps_per_second': 19.548}